In [ ]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

In [ ]:
import pandas as pd
taiwan_data = pd.read_csv("../ComputedDataV2/Accident/combined_data_in_taiwan.csv")
combined_data = pd.read_csv('../ComputedDataV2/ForModel/combined_data_with_hotspot.csv')

taiwan_data.shape, combined_data.shape

In [ ]:
roundabout = combined_data[combined_data['道路型態子類別名稱'] == '圓環']

roundabout_hotspot = combined_data[(combined_data['道路型態子類別名稱'] == '圓環') &
                                    (combined_data['道路類別-第1當事者-名稱'] == '市區道路') &
                                    (combined_data['速限-第1當事者'] < 60) &
                                    (combined_data['youbike_100m_count'] != 0)
                                    ]

In [ ]:
def categorize_roundabout(row):
    if (row['號誌-號誌種類名稱'] == '無號誌') & (row['車道劃分設施-分向設施大類別名稱'] == '無'):
        return 'No signal / No lane division'
    elif (row['號誌-號誌種類名稱'] == '無號誌') & (row['車道劃分設施-分向設施大類別名稱'] != '無'):
        return 'No signal / With lane division'
    elif (row['號誌-號誌種類名稱'] != '無號誌') & (row['車道劃分設施-分向設施大類別名稱'] == '無'):
        return 'With signal / No lane division'
    else:
        return 'With signal / With lane division'

roundabout['cross_roundabout'] = roundabout.apply(categorize_roundabout, axis=1)

In [ ]:
roundabout.to_csv('../ComputedDataV2/plot/roundabout_data_with_cross_type.csv', index=False)

In [ ]:
combined_data['當事者區分-類別-子類別名稱-車種'].unique()

In [ ]:
pedestrian = combined_data[combined_data['當事者區分-類別-子類別名稱-車種'] == '行人']
bike = combined_data[combined_data['當事者區分-類別-子類別名稱-車種'] == '腳踏自行車']
bike_ratio = combined_data[(combined_data['當事者區分-類別-子類別名稱-車種'] == '腳踏自行車') &
                           (combined_data['youbike_100m_count'] != 0)]

In [ ]:
import pydeck as pdk

# Define a layer to display on a map
layer = pdk.Layer(
    "HexagonLayer",
    combined_data[["經度", "緯度"]],
    get_position=["經度", "緯度"],
    auto_highlight=True,
    elevation_scale=50,
    # radius=1000,
    pickable=True,
    elevation_range=[0, 3000],
    extruded=True,
    coverage=1,
)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=120.533531,
    latitude=22.505826,
    zoom=6,
    min_zoom=5,
    max_zoom=15,
    pitch=40,
    bearing=0,
)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html("../ComputedDataV2/Deck/combined_data.html")